In [1]:
import pandas as pd
import numpy as np
import geopy.distance
import os
import sys

sys.path.append('..')
from common.base import session, engine

In [2]:
def distancer(row):
    coords_1 = (row['lat'], row['long'])
    coords_2 = (row['next_lat'], row['next_long'])
    return geopy.distance.distance(coords_1, coords_2).km

def create_dataframe():
    data = session.execute(
        '''
        SELECT datetime, trip_id, make, model, lat, long, velocity
        FROM drive
        INNER JOIN vehicle on vehicle.vehicle_spec_id = drive.vehicle_spec_id;'''
        ).all()

    df = pd.DataFrame(data, columns=['Datetime','trip_id','make','model','lat','long','velocity'])

    return df


In [3]:
df = create_dataframe()

#Manipulate data to get distance travelled using function Distancer 
df['next_lat'] = df.groupby('trip_id')['lat'].shift(1)
df['next_long'] = df.groupby('trip_id')['long'].shift(1)
df['next_lat'].fillna(df['lat'], inplace=True)
df['next_long'].fillna(df['long'], inplace=True)

In [5]:
df.shape

(355208, 9)

In [6]:
 #Change dataframe to get the difference between the timestamps
df['timestamp_end'] = df.groupby('trip_id')['Datetime'].shift(1)
df['trip_duration_minutes'] = (df['Datetime'] - df['timestamp_end'])/np.timedelta64(1,'m')
df['timestamp_end'].fillna(df['Datetime'], inplace=True)
df['trip_duration_minutes'].fillna(0,inplace=True)

In [7]:
df.shape

(355208, 11)

In [8]:
df['distance'] = df.apply(distancer, axis=1)

In [9]:
df.shape

(355208, 12)

In [10]:
conn = engine.connect()
df.to_sql('daily_trip_long_lat_calc', conn, if_exists='replace')
print("Dataframe lat and lon calculation exported to PSQL")

Dataframe lat and lon calculation exported to PSQL
